# Классификация и анализ поисковых запросов

### Импортируем необходимые библиотеки

In [1]:
import os.path #Для работы с файлами
import numpy as np #Для работы с массивами
import pandas as pd #Для работы с таблицами
from sklearn.model_selection import train_test_split #Для получения меньшего по размеру набора данных
from matplotlib import pyplot as plt; from matplotlib import dates #Для построения графиков
from datetime import datetime #Для работы с датой и временем
from gensim import corpora, models #Для обработки текстов
import pymorphy2 #Для работы с формами слов
import nltk #Для удаления стоп-слов (бесполезных слов)
from pandas.plotting import register_matplotlib_converters; register_matplotlib_converters() #Для построения графиков с датой
np.random.seed(1329) #Для воспроизводимости результатов
morph = pymorphy2.MorphAnalyzer() #Анализатор слов
import pyLDAvis; import pyLDAvis.gensim #Визуализация тем

### Выберем, с какой частью данных мы хотим работать. 
Large - 100%, medium - 10%, small - 1%. 
Генерация таблиц меньшего размера произойдет только после первого запуска программы.
Порядок времени работы программы с 1% данных - 10 минут, с 10% - 1.5 часа, 100% - сутки.

In [2]:
#Введите small, medium или large, чтобы обработать соответствующий вариант таблицы
print('Введите small, medium или large, чтобы обработать соответствующий вариант таблицы')
name = 'small'
#Переменная new равна True, если отсутствует таблица данного размера или нормализованная таблица данного размера
new = not (os.path.isfile(f'football_{name}.csv') and os.path.isfile(f'football_{name}_normalised.csv'))

Введите small, medium или large, чтобы обработать соответствующий вариант таблицы
medium


### Далее следует загрузка данных в скрипт

In [ ]:
#Загрузка данных, если они уже были сохранены в нормализованном виде
if not new:
    data = pd.read_csv(f'football_{name}_normalised.csv', header = None, names = ['Request', 'DateTime'])
    data['DateTime'] = pd.to_datetime(data['DateTime'])

In [3]:
#data = pd.read_csv(f'football_{name}.csv', header = None, names = ['Request', 'DateTime'])

In [ ]:
#Загрузка данных, если альтернативных таблиц еще нет
if new:
    data_raw = pd.read_csv('football')

In [ ]:
#Создание таблицы data с двумя столбцами - Request и DateTime, если альтернативных таблиц еще нет
if new:
    list_requests = [row[:-20] for row in data_raw['normal_query\tdatetime']]
    list_datetime = [datetime.strptime(row[-19:-9]+row[-8:], '%Y-%m-%d%H:%M:%S') for row in data_raw['normal_query\tdatetime']]
    data_large = pd.DataFrame({'Request':list_requests, 'DateTime':list_datetime})
    del list_requests
    del list_datetime

In [ ]:
#Создание меньших по объему таблиц данных, если их еще нет
if new:
    data_medium, smth = train_test_split(data_large, train_size = 0.1) #в 10 раз меньше данных
    data_small, smth = train_test_split(data_large, train_size = 0.01) #в 100 раз меньше данных
    if name=='large':
        data = data_large
    if name=='medium':
        data = data_medium
    if name=='small':
        data = data_small
    del smth

In [ ]:
#Сохранение таблиц в .csv файл для дальнейшей работы, если это еще не сделано
if new:
    data_large.to_csv('football_large.csv', header = False, index = False, encoding = 'utf-8')
    data_medium.to_csv('football_medium.csv', header = False, index = False, encoding = 'utf-8')
    data_small.to_csv('football_small.csv', header = False, index = False, encoding = 'utf-8')

In [ ]:
#Переведем дату и время в формат datetime
data['DateTime'] = pd.to_datetime(data['DateTime'])

### Данные загружены. Проведем лемматизацию, а так же оставим ТОЛЬКО глаголы и неодушевленные существительные

In [10]:
#Нормализуем формы слов - приводим все формы одного слова к единому виду, если это еще не сделано
#Переведем дату и время в формат datetime
data['Request'] = data['Request'].astype('str') #На всякий случай переведем все запросы в тип str
if new:
    data['DateTime'] = pd.to_datetime(data['DateTime'])
types = ['NOUN', 'VERB'] #Список допустимых частей речи
types2 = ['NOUN', 'VERB', 'ADJF', 'ADJS', 'COMP', 'INFN', 'PRTF', 'PRTS', 'GRND', 'ADVB'] #Список допустимых частей речи - расширенный
def normalise_sentence(sentence): #Функция, лемматизирующая предложение
    normal_sentence = ''
    for word in sentence.split(' '):
        p = morph.parse(word)[0]
        if ('NOUN' in p.tag and 'inan' in p.tag): #Отбираем только неодушивленные существительные
            normal_sentence+=f' {p.normal_form}'
        if 'VERB' in p.tag or 'INFN' in p.tag:
            normal_sentence+=f' {p.normal_form}' #Отбираем глаголы
    return normal_sentence

if new:
    data['Request'] = data['Request'].astype('str').apply(normalise_sentence)

In [37]:
if new:
    #Сохранение лемматизированной таблицы, если это еще не сделано
    data.to_csv(f'football_{name}_normalised.csv', header = False, index = False, encoding = 'utf-8')

### Создание словаря и корпуса gensim для последующего тематического моделирования

In [38]:
#Создание словаря и корпуса
data = data.fillna('') #Заполним пропуски пустыми строками
texts = [request.split(' ') for request in data['Request'].values]
dictionary = corpora.Dictionary(texts)
print('Введите минимальное число упоминаний слова. По умолчанию - 1500 для medium версии')
min_count = 60
dictionary.filter_extremes(no_below=min_count, no_above=0.1) #Убираем слишком редкие и слишком частые слова
common_words_to_delete = ['такой', 'какой'] #Сюда можно вручную добавить слова для удаления
del_ids = [k for k,v in dictionary.items() if v in common_words_to_delete or len(v)<=1] #Убираем лишние слова
dictionary.filter_tokens(bad_ids=del_ids)
corpus = [dictionary.doc2bow(text) for text in texts]

Введите минимальное число упоминаний слова. По умолчанию - 60


### Узнаем количество слов в получившемся словаре

In [39]:
print(len(dictionary.keys()))

794


### Обучение тематической модели LDA

In [40]:
#Обучаение тематической модели для n тем. Стандартно n = 20
print('Введите число тем. Стандартно - 15')
n = 15
print(f'n = {n}')
LDA = models.LdaModel(corpus, num_topics = n, id2word = dictionary, passes = 10, 
                      random_state = 1329, update_every=1) #chunksize=100

Введите число тем. Стандартно - 15
n = 15


### Посмотрим на получившиеся темы

In [41]:
#Вывод итоговых тем
LDA.show_topics(num_topics = n, num_words = 7, formatted = True)

[(0,
  '0.147*"видео" + 0.095*"телефон" + 0.056*"рецепт" + 0.051*"номер" + 0.038*"мультик" + 0.036*"текст" + 0.036*"язык"'),
 (1,
  '0.328*"порно" + 0.228*"год" + 0.059*"город" + 0.051*"мультфильм" + 0.043*"делать" + 0.031*"ростов" + 0.020*"дон"'),
 (2,
  '0.258*"скачать" + 0.194*"купить" + 0.068*"дом" + 0.044*"музыка" + 0.044*"торрент" + 0.035*"программа" + 0.024*"майнкрафт"'),
 (3,
  '0.080*"рождение" + 0.059*"новосибирск" + 0.048*"класс" + 0.041*"курс" + 0.040*"франция" + 0.040*"поздравление" + 0.040*"список"'),
 (4,
  '0.125*"секс" + 0.114*"карта" + 0.061*"прямая" + 0.054*"канал" + 0.049*"играть" + 0.048*"машина" + 0.044*"авто"'),
 (5,
  '0.102*"область" + 0.068*"почта" + 0.059*"сбербанк" + 0.052*"картинка" + 0.044*"петербург" + 0.043*"санкт" + 0.040*"вход"'),
 (6,
  '0.087*"день" + 0.083*"быть" + 0.077*"песня" + 0.066*"билет" + 0.057*"тв" + 0.046*"инстагра" + 0.037*"отдых"'),
 (7,
  '0.167*"москва" + 0.148*"цена" + 0.128*"игра" + 0.062*"яндекс" + 0.038*"новгород" + 0.025*"вода" + 

### Посмотрим на точность модели

In [42]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=LDA, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3901582280496848


### Визуализируем темы

In [43]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(LDA, corpus, dictionary, mds = 'mmds')
pyLDAvis.display(vis)

D:\Programs\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




### Построим распределение числа запросов на футбольную тему в день от даты

Слова на футбольную тему загружаются из текстового файла football_words.txt

In [ ]:
#Определим функцию, которая будет строить распределение.
#На вход она принимает pandas DataFrame со столбцами 'Request' и 'DateTime',
#а так же массив нужных нам слов и период времени, за который будет производиться подсчет. например '3D' - три дня, '6H' - 6 часов.
def plot_topic(data, topic, period):
    df = data
    counts = []
    for line in df['Request']:
        count = 0
        for word in topic:
            if word in line:
                count+=1
        counts.append(count)
    df['Count'] = counts
    df['date_minus_time'] = df["DateTime"].apply(lambda df : datetime(year = df.year, month = df.month,
                                                                      day = df.day, hour = df.hour)) 
    df.set_index(df["date_minus_time"], inplace = True)
    df_resampled = df['Count'].resample(period).sum()
    
    fig = plt.figure(figsize = (16, 9))
    ax = plt.axes()
    plt.plot_date(df_resampled.index, df_resampled.values, linestyle = 'solid', aa = True)
    plt.xlabel('Date')
    plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
    plt.ylabel('Related requests per day')
    plt.show()

In [ ]:
#Загрузим футбольные слова
football_words_file = open('football_words.txt', 'r')
content = football_words_file.read()
topic_football = content.split('   ')
#Построим график, используя функцию plot_topic
plot_topic(data, topic_football, '1H') #Подсчет будем вести каждый час

### Виден легкий тренд на понижение актуальности темы по мере приближения к концу ЧМ
### Локальные максимумы объясняются проведением важных матчей
Аналогичным образом можно получить график для любой темы.

### Напишем функцию, которая сопоставляет слову тему

In [45]:
def word_to_topic(word):
    word = normalise_sentence(word)[1:] #Нормализуем слово, воспользовавшись старой функцией. Отрежем лишний пробел в начале.
    if word not in dictionary.token2id: print(f'Error: {word} is not in dictionary.'); return None
    i = dictionary.token2id[word]
    result = LDA.get_term_topics(i)
    if result==[]: print(f'{word} is not frequent enough.'); return None
    report = f'{word} belongs to'
    topics = []
    for part in result:
        report+=f' topic №{part[0]};'
        report+=f' probability - {part[1]}'
        topics.append(part[0])
    print(report)
    return topics

### С помощью этой функции и можно организовать колдунщик
Пример использования:

In [46]:
word_to_topic('смотреть')

смотреть belongs to topic №13; probability - 0.25190383195877075


[13]